In [1]:
import sys
sys.path.append('..')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math
import subprocess
import os

In [2]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.188.123'
ROBOT_PORT = 30004
config_filename = 'pose_pilot_configuration_without_button.xml'

keep_running = True
logging.getLogger().setLevel(logging.INFO)

In [3]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')

In [4]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)

INFO:root:Controller version: 3.5.4.0


In [5]:
process = subprocess.Popen(r'C:\OpenHaptics\Developer\3.4.0\examples\HD\console\getTouchState\x64\Release\getTouchState.exe',
    stdin = subprocess.PIPE,
    stdout = subprocess.PIPE, 
    stderr = subprocess.PIPE,
    universal_newlines = True
    )

In [6]:
process.stdout.readline()
process.stdout.readline()
process.stdout.readline()

'\n'

In [7]:
def getTouchState(p):
    p.stdin.write('-')
    p.stdin.flush()
    state = process.stdout.readline()
    return [float(i) for i in state.split(' ')]

In [8]:
class transform:
    def __init__(self, scale, offset):
        assert len(scale) == len(offset)
        self.scale = scale
        self.offset = offset
    
    def doTransform(self, p):
        return [s*x+b for s, x , b in zip(self.scale, p, self.offset)]

In [9]:
class deltaTransform:
    def __init__(self, original_base, scale, target_base):
        self.o_base = original_base
        self.scale = scale
        self.t_base = target_base
        
    def doTransform(self, original_point):
        # scale and original point delta
        scaled_point = [s*(o-b) for s, b , o in zip(self.scale, self.o_base, original_point)]
        # transform cartesians
        # TODO: make it general, this only works to swap y and z
        scaled_point[1], scaled_point[2] = -scaled_point[2], scaled_point[1]
        
        t_point = [t+s for t, s in zip(self.t_base, scaled_point)]
        return t_point

In [10]:
t = deltaTransform([1, 1, 1], [2, 2, 2], [1, 2, 3])
t.doTransform([2, 2, 3]) # [3, -2, 5]

[3, -2, 5]

In [11]:
def setp_to_list(setp):
    list = []
    for i in range(0,6):
        list.append(setp.__dict__["input_double_register_%i" % i])
    return list

def list_to_setp(setp, list):
    for i in range (0,6):
        setp.__dict__["input_double_register_%i" % i] = list[i]
    return setp

In [12]:
setp.input_double_register_0 = 0
setp.input_double_register_1 = 0
setp.input_double_register_2 = 0
setp.input_double_register_3 = 0
setp.input_double_register_4 = 0
setp.input_double_register_5 = 0
  
# The function "rtde_set_watchdog" in the "rtde_control_loop.urp" creates a 1 Hz watchdog
watchdog.input_int_register_0 = 0

T = 5 #sec, Time constant
start_time = time.time()

robot_base = [0.13, -0.61, 0.30, 0.60, -3.0, -0.07] # communication to robot is in meters and radians
controller_base = [1.60547, -97.9115, -85.3162, 0.0812017, -0.556647, 0.742585] # communication with controller is in mm and rad

#scale = [0.0005, 0.0005, 0.0005, 1, 1, 1]
#scale = [0, 0, 0, 0, 0, 0]
scale = [0.002, 0.002, 0.002, 1, 1, 1]
#offset = [r-c*s for r, c, s in zip(robot_base, controller_base, scale)]
#offset[3:6] = [0, 0, 0]

#tr = transform(scale, offset)
tr = deltaTransform(controller_base, scale, robot_base)
# [0.127489254, -0.69790714, 0.3223092, 0.6, -3.0, -0.07]

In [13]:
new_setp = getTouchState(process)
tr.doTransform(new_setp[0:6])

[0.07594746, -0.6189046, 0.3053742, 0.52589167, -2.930899, -2.535605]

In [14]:
#start data synchronization
if not con.send_start():
    sys.exit()

#while time.time() - start_time < 15:
#    current_rad = math.pi/2
#    new_setp = [c + a*math.sin(current_rad) for a,c in zip(amp, center)]
#    list_to_setp(setp, new_setp)
    
    # send new setpoint        
#    con.send(setp)
past_button_state = False
# control loop
while keep_running:
    # receive the current state
    state = con.receive()
    
    if state is None:
        break;
    
    # Calc new setp
    new_setp = getTouchState(process)
    new_setp = tr.doTransform(new_setp[0:6])
    list_to_setp(setp, new_setp)
    
    # send new setpoint        
    con.send(setp)
    
    # kick watchdog
    con.send(watchdog)

INFO:root:RTDE synchronization started
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
I

INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:

INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:skipping package(1)
INFO:root:lost connection with controller
INFO:root:RTDE disconnected


In [15]:
con.send_pause()
con.disconnect()
process.terminate() 

ERROR:root:Unable to send: not connected to Robot
ERROR:root:RTDE synchronization failed to pause
